In [5]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

## 딥러닝 모델

In [6]:
class NeuralNet(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet, self).__init__()
        # 사용할 Layers
        self.mlp1 = layers.Dense(hidden_1, activation=tf.nn.relu) # 첫번째 Layer
        self.mlp2 = layers.Dense(hidden_2, activation="relu") # 둘 다 사용 가능
        self.mlp3 = layers.Dense(num_classes) # actiavtion="softmax"

    # tensorflow에서는 forward가 아닌 call을 사용
    def call(self, x):
        print('x:',x.shape)
        x = self.mlp1(x) # 첫 레이어를 거친 값
        print('x2:',x.shape)
        x = self.mlp2(x)
        print('x3:',x.shape)
        x = self.mlp3(x)
        print('x4:',x.shape)
        x = tf.nn.softmax(x)
        return x

In [7]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

* 모델 선언

In [8]:
model = NeuralNet(hidden_1, hidden_2, num_classes)

* 임의의 데이터로 모델 적용

In [10]:
data = tf.random.normal(shape=(1,28*28), mean=0., stddev=1.)
print('data:', data.shape)
print('-'*40)

pred = model(data) # softmax를 거쳐 0과 1 사이의 값
print('pred:', pred)
print('-'*40)

y_hat = tf.argmax(pred, 1) # 최대값인 하나를 인덱스로
print('y hat:', y_hat)

data: (1, 784)
----------------------------------------
x: (1, 784)
x2: (1, 128)
x3: (1, 256)
x4: (1, 10)
pred: tf.Tensor(
[[0.1029661  0.04044169 0.09568977 0.11849261 0.04040658 0.23408753
  0.15327367 0.03012977 0.09202974 0.09248251]], shape=(1, 10), dtype=float32)
----------------------------------------
y hat: tf.Tensor([5], shape=(1,), dtype=int64)


### layers.Dense

In [11]:
x = tf.random.normal(shape=(3,28,28), mean=0., stddev=1.)

Without activation function

In [13]:
fc_layer = layers.Dense(100) # hidden layer의 size가 100
x_after_fc = fc_layer(tf.reshape(x,(-1,28*28))) # Fully Connected Layer는 1D만 받음
print('x_after_fc:',x_after_fc.shape)
print(x_after_fc[0][:10])

x_after_fc: (3, 100)
tf.Tensor(
[ 0.5234126   0.62165135  2.0812209   0.0570631   0.09281635  0.04693154
 -1.533257   -1.5113839  -1.2004786  -0.47020486], shape=(10,), dtype=float32)


With activation function

In [19]:
fc_layer = layers.Dense(100, activation='relu') # ReLU => 음수는 모두 0으로
x_after_fc = fc_layer(tf.reshape(x,(-1,28*28)))
print('x_after_fc:',x_after_fc.shape)
print(x_after_fc[0][:10])

x_after_fc: (3, 100)
tf.Tensor(
[0.         0.         0.         0.11074781 0.9276475  0.2737847
 0.9741174  0.         0.6939348  0.        ], shape=(10,), dtype=float32)


### Softmax

In [20]:
x = tf.random.normal(shape=(1,10), mean=0., stddev=1.)
x

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 0.52438647, -0.7313995 , -0.17398156,  1.3262413 ,  0.8385133 ,
        -0.08098479, -0.17513578, -1.5020161 ,  0.2095927 , -0.99065006]],
      dtype=float32)>

In [21]:
pred = tf.nn.softmax(x)
print('pred:',pred)
print('sum of pred:',tf.reduce_sum(pred))

pred: tf.Tensor(
[[0.13324046 0.0379538  0.06627333 0.29708263 0.18241465 0.0727322
  0.06619688 0.01756227 0.09725747 0.02928628]], shape=(1, 10), dtype=float32)
sum of pred: tf.Tensor(1.0, shape=(), dtype=float32)


### tf.keras.Sequential

In [22]:
class NeuralNet(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = layers.Dense(hidden_1, activation=tf.nn.relu)
        self.fc2 = layers.Dense(hidden_2, activation=tf.nn.relu)
        self.out = layers.Dense(num_classes)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        x = tf.nn.softmax(x)
        return x

* Hyperparameter : 사람이 정의해 주어야 하는 변수

In [23]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

model = NeuralNet(hidden_1, hidden_2, num_classes)

* sequantial : 한번에 처리하기

In [24]:
class NeuralNet2(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet2, self).__init__() # 상속
        sequential = tf.keras.Sequential([
            layers.Dense(hidden_1, activation=tf.nn.relu),
            layers.Dense(hidden_2, activation=tf.nn.relu),
            layers.Dense(num_classes, activation=tf.nn.softmax),
        ])

    def call(self, x):
        x = sequential(x)
        return x

In [25]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

model = NeuralNet2(hidden_1, hidden_2, num_classes)

### Sequential + add

* sequential을 먼저 선언하고 Layer를 추가해도 됨

In [26]:
model = tf.keras.Sequential()

In [27]:
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### Compile

* compile() : 모델을 기계가 이해할 수 있도록 컴파일 하는 것 => 손실 함수와 옵티마이저, 메트릭 함수를 선택
  * optimizer = 훈련 과정을 설정하는 옵티마이저를 설정합니다.
  * loss = 훈련 과정에서 사용할 손실 함수(loss function)를 설정합니다.
  * metrics = 훈련을 모니터링하기 위한 지표를 선택합니다.

In [28]:
model.compile(loss='categorical_crossentropy', # labels가 one-hot일 경우 사용
              optimizer='sgd',
              metrics=['accuracy'])

In [29]:
model.compile(loss='sparse_categorical_crossentropy', # labels가 integer일 경우 사용
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), # learning rate 설정
              metrics=['accuracy'])